# Topological optimization of binary structures (TOBS)

### Description

The method of topological optimization of binary structures ([TOBS](https://www.sciencedirect.com/science/article/abs/pii/S0168874X17305619?via%3Dihub)) was originally developed in the context of optimal distribution of material in mechanical components. In its core, is a heuristic to solve binary optimization problems by first linearizing the objective and constraints. Then, a binary nonlinear program is solved (default solver is [Cbc](https://github.com/jump-dev/Cbc.jl)) to determine which binary variables must be flipped in the current iteration.

### Packages and parameters

In [1]:
using NonconvexTOBS, TopOpt
E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force
rmin = 6.0 # filter radius
xmin = 0.001 # minimum density
V = 0.5 # maximum volume fraction
p = 3.0 # topological optimization penalty

3.0

### Define FEA problem

In [2]:
problem_size = (160, 100); # size of rectangular mesh
x0 = fill(1.0, prod(problem_size)); # initial design
problem = PointLoadCantilever(Val{:Linear}, problem_size, (1.0, 1.0), E, v, f);

FEA solver and auxiliary functions need to be defined as well:

In [3]:
solver = FEASolver(Direct, problem; xmin=xmin);
cheqfilter = DensityFilter(solver; rmin=rmin); # filter function
comp = TopOpt.Compliance(solver); # compliance function

The usual topology optimization problem adresses compliance minimization under volume restriction. Therefore, the objective and the constraint are:

In [4]:
obj(x) = comp(cheqfilter(PseudoDensities(x))); # compliance objective
constr(x) = sum(cheqfilter(PseudoDensities(x))) / length(x) - V; # volume fraction constraint

### Optimization setup

Finally, the optimization problem is defined and solved:

In [5]:
m = Model(obj); # create optimization model
addvar!(m, zeros(length(x0)), ones(length(x0))); # setup optimization variables
Nonconvex.add_ineq_constraint!(m, constr); # setup volume inequality constraint
options = TOBSOptions(); # optimization options with default values
TopOpt.setpenalty!(solver, p);

Perform TOBS optimization

In [6]:
@time r = Nonconvex.optimize(m, TOBSAlg(), x0; options=options)

[ Info: iter = 1, obj = 21.833, constr_vio_norm = 0.5, er = 1.0
[ Info: iter = 2, obj = 22.37, constr_vio_norm = 0.45, er = 1.0
[ Info: iter = 3, obj = 23.35, constr_vio_norm = 0.405, er = 1.0
[ Info: iter = 4, obj = 24.327, constr_vio_norm = 0.364, er = 1.0
┌ Warning: Subproblem is infeasible. Temporarily relaxing the subproblem.
└ @ NonconvexTOBS ~/.julia/packages/NonconvexTOBS/V1xTL/src/NonconvexTOBS.jl:114
┌ Warning: Subproblem is infeasible. Temporarily relaxing the subproblem.
└ @ NonconvexTOBS ~/.julia/packages/NonconvexTOBS/V1xTL/src/NonconvexTOBS.jl:114
┌ Warning: Subproblem is infeasible. Temporarily relaxing the subproblem.
└ @ NonconvexTOBS ~/.julia/packages/NonconvexTOBS/V1xTL/src/NonconvexTOBS.jl:114
[ Info: iter = 8, obj = 26.303, constr_vio_norm = 0.328, er = 1.0
┌ Warning: Subproblem is infeasible. Temporarily relaxing the subproblem.
└ @ NonconvexTOBS ~/.julia/packages/NonconvexTOBS/V1xTL/src/NonconvexTOBS.jl:114
┌ Warning: Subproblem is infeasible. Temporarily relaxi

NonconvexTOBS.TOBSResult{Vector{Float64}, Float64, Float64}([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 41.728710072074776, 0.04243829893664462)

### Results

In [7]:
@show obj(r.minimizer)
@show constr(r.minimizer)
topology = r.minimizer;

obj(r.minimizer) = 41.728710072074776
constr(r.minimizer) = 4.168905620716146e-8


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*